<a href="https://colab.research.google.com/github/shenzhun/machine-learning-prep/blob/master/tensorflow/day21_tree_ways_of_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras import *

In [61]:
data_path = '/content/drive/MyDrive/InterviewPrep/ML/eat_tensorflow2_in_30_days/data'

In [62]:
train_token_path = data_path + "/imdb/train_token.csv"
test_token_path = data_path +  "/imdb/test_token.csv"

MAX_WORDS = 10000  # We will only consider the top 10,000 words in the dataset
MAX_LEN = 200  # We will cut reviews after 200 words
BATCH_SIZE = 20 

# Constructing data pipeline
def parse_line(line):
    t = tf.strings.split(line,"\t")
    label = tf.reshape(tf.cast(tf.strings.to_number(t[0]),tf.int32),(-1,))
    features = tf.cast(tf.strings.to_number(tf.strings.split(t[1]," ")),tf.int32)
    return (features,label)

ds_train=  tf.data.TextLineDataset(filenames = [train_token_path]) \
   .map(parse_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

ds_test=  tf.data.TextLineDataset(filenames = [test_token_path]) \
   .map(parse_line,num_parallel_calls = tf.data.experimental.AUTOTUNE) \
   .shuffle(buffer_size = 1000).batch(BATCH_SIZE) \
   .prefetch(tf.data.experimental.AUTOTUNE)

Modeling Using Sequential

In [63]:
tf.keras.backend.clear_session()

model = models.Sequential()

model.add(layers.Embedding(MAX_WORDS, 7, input_length=MAX_WORDS))
model.add(layers.Conv1D(filters=64, kernel_size=5, activation="relu"))
model.add(layers.MaxPool1D(2))
model.add(layers.Conv1D(filters=32, kernel_size=3, activation="relu"))
model.add(layers.MaxPool1D(2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation="sigmoid"))

model.compile(optimizer='Nadam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'AUC'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10000, 7)          70000     
_________________________________________________________________
conv1d (Conv1D)              (None, 9996, 64)          2304      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 4998, 64)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 4996, 32)          6176      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2498, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 79936)             0         
_________________________________________________________________
dense (Dense)                (None, 1)                 7

In [64]:
import datetime
baselogger = callbacks.BaseLogger(stateful_metrics=['AUC'])
logdir = data_path + '/keras_model/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(ds_train, validation_data=ds_test, epochs=6, callbacks=[baselogger, tensorboard_callback])

Epoch 1/6


NotFoundError: ignored

In [44]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt

def plot_metric(history, metric):
  train_metrics = history.history[metric]
  val_metrics = history.history['val_'+metric]

  epochs = range(1, len(train_metrics)+1)
  plt.plot(epochs, train_metrics, 'bo--')
  plt.plot(epochs, val_metrics, 'ro--')

  plt.title('Training and validation '+metric)
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend(["train_"+metric, "val_"+metric])
  plt.show()


In [45]:
plot_metric(history, "AUC")

NameError: ignored

Modeling Using Functional API

In [46]:
tf.keras.backend.clear_session()

inputs = layers.Input(shape=[MAX_LEN])
x = layers.Embedding(MAX_WORDS, 7)(inputs)

branch1 = layers.SeparableConv1D(64, 3, activation="relu")(x)
branch1 = layers.MaxPool1D(3)(branch1)
branch1 = layers.SeparableConv1D(32, 3, activation="relu")(branch1)
branch1 = layers.GlobalMaxPool1D()(branch1)

branch2 = layers.SeparableConv1D(64, 5, activation="relu")(x)
branch2 = layers.MaxPool1D(5)(branch2)
branch2 = layers.SeparableConv1D(32, 5, activation="relu")(branch2)
branch2 = layers.GlobalMaxPool1D()(branch2)

branch3 = layers.SeparableConv1D(64, 7, activation="relu")(x)
branch3 = layers.MaxPool1D(7)(branch3)
branch3 = layers.SeparableConv1D(32, 7, activation="relu")(branch3)
branch3 = layers.GlobalMaxPool1D()(branch3)

concat = layers.Concatenate()([branch1, branch2, branch3])
outputs = layers.Dense(1, activation="sigmoid")(concat)

model = models.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='Nadam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'AUC'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 7)       70000       input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv1d (SeparableConv (None, 198, 64)      533         embedding[0][0]                  
__________________________________________________________________________________________________
separable_conv1d_2 (SeparableCo (None, 196, 64)      547         embedding[0][0]                  
______________________________________________________________________________________________

In [47]:
import datetime
logdir = "../data/keras_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, historgram_freq=1)
history = model.fit(ds_train, validation_data=ds_test, epochs=6, callbacks=[tensorboard_callback])

ValueError: ignored

In [ ]:
plot_metrics(history, 'AUC')

Customized Modeling Using Child Class Of Model

In [51]:
class ResBlock(layers.Layer):
  def _init__(self, kernel_size, **kargs):
    super(ResBlock, self).__init__(**kargs)
    self.kernel_size = kernel_size
  
  def build(self, input_shape):
    self.conv1 = layers.Conv1D(filter=64, 
                               kernel_size=self.kernel_size,
                               activations="relu",
                               padding="same")
    self.conv2 = layers.Conv1D(filter=32,
                               kernel_size=self.kernel_size,
                               activations="relu",
                               padding="same")
    self.conv3 = layers.Conv1D(filter=input_shape[-1],
                               kernel_size=self.kernel_size,
                               activations="relu",
                               padding="same")
    self.maxpool = layers.MaxPool1D(2)
    super(ResBlock, self).build(input_shape)

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.conv2(x)
    x = self.conv3(x)
    x = layers.Add()([inputs, x])
    s = self.maxpool(x)
    return x
  
  def get_config(self):
    config = super(ResBlock,self).get_config()
    config.update({'kernel_size': self.kernel_size})
    return config

In [52]:
resblock = ResBlock(kernel_size = 3)
resblock.build(input_shape(None, 200, 7))
resblock.compute_output_shape(input_shape=(None, 200, 7))

TypeError: ignored

In [54]:
class ImdbModel(models.Model):
  def __init__(self):
    super(ImdbModel, self).__init__()
  
  def build(self, input_shape):
    self.embedding = layers.Embedding(MAX_WORDS, 7)
    self.block1 = ResBlock(7)
    self.block2 = ResBlock(5)
    self.dense = layers.Dense(1, activations="sigmoid")
    super(ImdbModel, self).build(input_shape)
  
  def call(self, x):
    x = self.embedding(x)
    x = self.block1(x)
    x = self.block2(x)
    x = layers.Flatten()(x)
    x = self.dense(x)
    return x

In [55]:
tf.keras.backend.clear_session()

model = ImdbModel()
model.build(input_shape=(None, 200))
model.summary()

model.compile(optimizer='Nadam',
              loss='binary_crossentropy',
              metrics=['accuracy', 'AUC'])

TypeError: ignored

In [56]:
import datetime

logdir = "../tflogs/keras_model/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
history = model.fit(ds_train, validation_data=ds_test,
                    epochs=6, callbacks=[tensorboard_callback])

RuntimeError: ignored

In [57]:
plot_metric(history, "AUC")

NameError: ignored